# Drzewa decyzyjne

In [147]:
import csv
import math
import numpy as np
from treelib import Node, Tree


In [148]:
def count_items(lista, calc_probability=False):
    d = {}
    for elem in lista:
        key = elem[-1]
        if d.get(key) is None:
            d[key] = 0
        d[key]+=1
    if calc_probability:
        for key in d:
            d[key]=d[key]/len(lista)
    return d

def enthropy(prob_dikt):
    ent = 0
    for key in prob_dikt:
        ent += prob_dikt[key] * math.log(prob_dikt[key],2)
    return -ent

def split(table, idx):
    d = {}
    if idx == 3: #jezeli po wieku
        temp = {}
        for passanger in table:
            age = float(passanger[idx])
            if temp.get(age) is None:
                temp[age] = []
            temp[age].append(passanger)
        temp = sorted(temp.items())
        tresshold_candidates = []
        last_state = temp[0][1][0][-1]
        for i, age in enumerate(temp):
            real_age = age[0]
            list_of_passangers_at_age = age[1]
            #print(age)
            if i == 0:
                continue
            probability = count_items(list_of_passangers_at_age, True)
            if probability.get('0', 0) == 1.0 or probability.get('1', 0) == 1.0:#obecny czysty podzbior
                if probability.get(last_state, 0) == 1.0:#ten sam stan jak poprzednio
                    continue
                else:#inny stan niz poprzednio stawiamy thresshold pomiędzy 
                    tresshold_candidates.append((float(real_age)+float(temp[i-1][0]))/2)
                    last_state = '0' if last_state == '1' else '1'
            else:# obecny mieszany podzbior stawiamy thresshold pomiędzy na wszelki wypadek - to tylko kandydat
                tresshold_candidates.append((float(real_age)+float(temp[i-1][0]))/2)
                last_state = 'MIX'
        #wybranie najlepszego thressholda
        best_gain_ratio = -1
        best_tresshold = None
        #print("Tresshold candidates:", tresshold_candidates)
        if len(tresshold_candidates) == 0:
            return { "ALL": table }
        for tresshold in tresshold_candidates:
            less_than_tresshold = []
            greater_equal_tresshold = []
            for passanger in table:
                if float(passanger[idx]) < float(tresshold):
                    less_than_tresshold.append(passanger)
                else:
                    greater_equal_tresshold.append(passanger)
            g_ratio = gain_ratio_for_split([less_than_tresshold, greater_equal_tresshold], table)
            if g_ratio > best_gain_ratio:
                best_gain_ratio = g_ratio
                best_tresshold = tresshold
        #podzial na dwa podzbiory wedlug najlepszego thressholda
        #print("Best tresshold:", best_tresshold)
        for passanger in table:
            #print("Passanger age:", passanger[idx])
            if float(passanger[idx]) < float(best_tresshold):
                key = "<"+str(best_tresshold)
            else:
                key = ">="+str(best_tresshold)
            if d.get(key) is None:
                d[key] = []
            d[key].append(passanger)
        return d
    #zwykly podzial przy wartościach dyskretnych
    for passanger in table:
        key = passanger[idx]
        if d.get(key) is None:
            d[key] = []
        d[key].append(passanger)
    return d

def conditional_enthropy(table, idx):
    splitted = split(table, idx)
    s = len(table)
    # print(s)
    cond_ent = 0
    for key in splitted:
        podzbior = splitted[key]
        sj = len(podzbior)
        # print(sj)
        ent = enthropy(count_items(podzbior,True))
        # print(ent)
        cond_ent += sj*ent/s
    return cond_ent

def gain(table, idx):
    cond_ent = conditional_enthropy(table,idx)
    ent = enthropy(count_items(table,True))
    return ent - cond_ent

def intrinsic_info(table, idx):
    splitted = split(table, idx)
    s = len(table)
    # print(s)
    cond_ent = 0
    for key in splitted:
        podzbior = splitted[key]
        sj = len(podzbior)
        # print(sj)
        lOg = math.log(sj/s,2)
        # print(ent)
        cond_ent += sj*lOg/s
    return -cond_ent

def gain_ratio(table, idx):
    g = gain(table, idx)
    intr = intrinsic_info(table, idx)
    return 0 if intr == 0 else g / intr

def gain_ratio_for_split(subsets, full_table):
    s = len(full_table)
    ent = enthropy(count_items(full_table, True))
    cond_ent = 0
    split_info = 0
    for subset in subsets:
        sj = len(subset)
        if sj == 0:
            continue
        prob = count_items(subset, True)
        cond_ent += sj * enthropy(prob) / s
        split_info += sj * math.log(sj / s, 2) / s
    gain = ent - cond_ent
    return 0 if split_info == 0 else gain / -split_info

def ID3(table, depth=0):
    best_attr = 0
    best_gain_ratio = -1
    for attr in range(len(table[0])-1):
        g_ratio = gain_ratio(table, attr)
        #print("  " * depth + f"Atrybut: {attr}, Gain ratio: {g_ratio:.4f}")
        if g_ratio > best_gain_ratio:
            best_gain_ratio = g_ratio
            best_attr = attr
    if best_gain_ratio == 0:
        class_counts = count_items(table)
        print("  " * depth + f"Leaf: {class_counts}")
        return
    print("  " * depth + f"Najlepszy atrybut do podziału: {best_attr}, Gain ratio: {best_gain_ratio:.4f}")
    split_table = split(table, best_attr)
    for key in split_table:
        print("  " * depth + f"Podzbior dla klucza: {key}")
        ID3(split_table[key], depth+1)



In [149]:
with open('titanic-homework.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    table = []
    for row in spamreader:
        if row[0]==('PassengerId'):
            continue
        table.append([item for i, item in enumerate(row) if i!=2 and i!=3])
        print(table[-1])
    
    # pro = count_items(table, True)
    # print(pro)
    # ent = enthropy(pro)
    # print("Entropia początkowa", ent)
    # cond_ent = conditional_enthropy(table,1)
    # print("Entropia warunkowa po podziale wg 1 atrybutu", cond_ent)
    # print("Gain po podziale wg 1 atrybutu", gain(table,1))
    # print("Gain ratio po podziale wg 1 atr", gain_ratio(,1,))
    

['1', '3', 'male', '22', '1', '0', '0']
['2', '1', 'female', '38', '1', '0', '1']
['3', '3', 'female', '26', '0', '0', '1']
['4', '1', 'female', '35', '1', '0', '1']
['5', '3', 'male', '35', '0', '0', '0']
['6', '3', 'male', '34', '0', '0', '0']
['7', '1', 'male', '54', '0', '0', '0']
['8', '3', 'male', '2', '3', '1', '0']
['9', '3', 'female', '27', '0', '2', '1']
['10', '2', 'female', '14', '1', '0', '1']
['11', '3', 'female', '4', '1', '1', '1']
['12', '1', 'female', '58', '0', '0', '1']
['13', '3', 'male', '20', '0', '0', '0']
['14', '3', 'male', '39', '1', '5', '0']
['15', '3', 'female', '14', '0', '0', '1']
['16', '2', 'female', '55', '0', '0', '1']
['17', '3', 'male', '2', '4', '1', '0']
['18', '2', 'male', '12', '0', '0', '1']
['19', '3', 'female', '31', '1', '0', '0']
['20', '3', 'female', '63', '0', '0', '1']
['21', '2', 'male', '35', '0', '0', '0']
['22', '2', 'male', '34', '0', '0', '0']
['23', '3', 'female', '15', '0', '0', '1']
['24', '1', 'male', '28', '0', '0', '1']
['25

In [150]:
ID3(table)

Najlepszy atrybut do podziału: 2, Gain ratio: 0.4032
Podzbior dla klucza: male
  Najlepszy atrybut do podziału: 3, Gain ratio: 0.4355
  Podzbior dla klucza: >=1.5
    Najlepszy atrybut do podziału: 0, Gain ratio: 0.0806
    Podzbior dla klucza: 1
      Leaf: {'0': 1}
    Podzbior dla klucza: 5
      Leaf: {'0': 1}
    Podzbior dla klucza: 6
      Leaf: {'0': 1}
    Podzbior dla klucza: 7
      Leaf: {'0': 1}
    Podzbior dla klucza: 8
      Leaf: {'0': 1}
    Podzbior dla klucza: 13
      Leaf: {'0': 1}
    Podzbior dla klucza: 14
      Leaf: {'0': 1}
    Podzbior dla klucza: 17
      Leaf: {'0': 1}
    Podzbior dla klucza: 18
      Leaf: {'1': 1}
    Podzbior dla klucza: 21
      Leaf: {'0': 1}
    Podzbior dla klucza: 22
      Leaf: {'0': 1}
    Podzbior dla klucza: 24
      Leaf: {'1': 1}
    Podzbior dla klucza: 27
      Leaf: {'0': 1}
    Podzbior dla klucza: 28
      Leaf: {'0': 1}
    Podzbior dla klucza: 30
      Leaf: {'0': 1}
    Podzbior dla klucza: 31
      Leaf: {'1': 1}
 

## Komentarz




- Aleksander Kamiński 155840
- Piotr Nowacki 15